## integrating data generators with batches to avoid memory issue and trying to fiting this model


In [1]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, Dense, Dropout, Input, LSTM, concatenate, RepeatVector, Embedding
from keras.optimizers import RMSprop
from os import listdir


#### some issues occuring solved in below cell

In [2]:

dirnamee=r'../../Screenshot-to-code/Bootstrap/resources/datasets/'

# def load_doc(file):
#     with open(file, 'r') as f:
#         text = f.read()
#     return text

# def load_data(dirname):
#     texts = []
#     images = []

#     # loading all the filenames
#     allfiles = listdir(dirname)
#     allfiles.sort()
#     for filename in allfiles:
#         if str(filename).endswith('npz'):
#             image = np.load(dirname + filename)
#             # images
#             images.append(image['features']/255.0)
#         else:
#             # text
#             syntax = '<START> ' + load_doc(dirname + filename) + ' <END>'
#             syntax = " ".join(syntax.split())
#             syntax = syntax.replace(',', ' ,')
#             texts.append(syntax)

#     return np.array(images), texts

# # Load the bootstrap tokens
# tokenizer = Tokenizer(filters='', split=" ", lower=False)
# tokenizer.fit_on_texts([load_doc('bootstrap.vocab')])

# # Add one spot for the empty word in the vocabulary 
# vocab_size = len(tokenizer.word_index) + 1

# # Map the input sentences into the vocabulary indexes
# train_sequences = tokenizer.texts_to_sequences(load_data(dirname=dirnamee)[1])

# # The longest set of bootstrap tokens
# max_sequence = max(len(s) for s in train_sequences)

# # Specify how many tokens to have in each input sentence
# max_length = 48

# def preprocess_data(sequences, features):
#     X, y, image_data = [], [], []
#     for img_no, seq in enumerate(sequences):
#         for i in range(1, len(seq)):
#             # Add the sentence until the current count(i) and add the current count to the output
#             in_seq, out_seq = seq[:i], seq[i]
#             # Pad all the input token sentences to max_sequence
#             in_seq = pad_sequences([in_seq], maxlen=max_sequence, padding='pre', truncating='post')[0]
#             # Turn the output into one-hot encoding
#             out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
#             # Add the corresponding image to the bootstrap token file
#             image_data.append(features[img_no])
#             # Cap the input sentence to 48 tokens and add it
#             X.append(in_seq[-max_length:])
#             y.append(out_seq)
#     return np.array(X, dtype='float16'), np.array(y, dtype='float16'), np.array(image_data, dtype='float16')

# # Data generator
# def data_generator(dirname, batch_size=32):
#     images, texts = load_data(dirname)
#     sequences = tokenizer.texts_to_sequences(texts)
    
#     while True:
#         # Shuffle the data
#         indices = np.arange(len(images))
#         np.random.shuffle(indices)
#         images = images[indices]
#         sequences = [sequences[i] for i in indices]
        
#         for start in range(0, len(images), batch_size):
#             end = min(start + batch_size, len(images))
            
#             X_batch, y_batch, image_data_batch = preprocess_data(sequences[start:end], images[start:end])
            
#             yield [image_data_batch, X_batch], y_batch


## issues solved

In [3]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, Dense, Dropout, Input, LSTM, concatenate, RepeatVector, Embedding
from keras.optimizers import RMSprop
from os import listdir
from sklearn.model_selection import train_test_split  # Added import for train-test split
dirnamee=r'../../Screenshot-to-code/Bootstrap/resources/datasets/'


def load_doc(file):
    with open(file, 'r') as f:
        text = f.read()
    return text

# Load data from the specified directory
def load_data(dirname):
    texts = []
    images = []

    # Loading all the filenames
    allfiles = listdir(dirname)
    allfiles.sort()
    for filename in allfiles:
        if str(filename).endswith('npz'):
            image = np.load(dirname + filename)
            # Images
            images.append(image['features'] / 255.0)
        else:
            # Text
            syntax = '<START> ' + load_doc(dirname + filename) + ' <END>'
            syntax = " ".join(syntax.split())
            syntax = syntax.replace(',', ' ,')
            texts.append(syntax)

    return np.array(images), texts

# Load the bootstrap tokens
tokenizer = Tokenizer(filters='', split=" ", lower=False)
tokenizer.fit_on_texts([load_doc('bootstrap.vocab')])

# Add one spot for the empty word in the vocabulary
vocab_size = len(tokenizer.word_index) + 1

# Map the input sentences into the vocabulary indexes
train_sequences = tokenizer.texts_to_sequences(load_data(dirname=dirnamee)[1])

# The longest set of bootstrap tokens
max_sequence = max(len(s) for s in train_sequences)

# Specify how many tokens to have in each input sentence
max_length = 48

def preprocess_data(sequences, features):
    X, y, image_data = [], [], []
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_sequence, padding='pre', truncating='post')[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Add the corresponding image to the bootstrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            X.append(in_seq[-max_length:])
            y.append(out_seq)
    return np.array(X, dtype='float16'), np.array(y, dtype='float16'), np.array(image_data, dtype='float16')


In [16]:

# Data generator with validation split
def data_generator(dirname, batch_size=32):
    images, texts = load_data(dirname)
    sequences = tokenizer.texts_to_sequences(texts)
    
    while True:
        # Shuffle the data
        indices = np.arange(len(images))
        np.random.shuffle(indices)
        images = images[indices]
        sequences = [sequences[i] for i in indices]
        
        # Calculate the split point for validation data (20%)
        split_point = int(0.2 * len(images))
        val_images, val_sequences = images[:split_point], sequences[:split_point]
        
        for start in range(split_point, len(images), batch_size):
            end = min(start + batch_size, len(images))
            
            X_batch, y_batch, image_data_batch = preprocess_data(sequences[start:end], images[start:end])
            
            yield [image_data_batch, X_batch], y_batch




# Train the model

In [17]:
# Data generator with validation split
def val_generator(dirname, batch_size=32):
    images, texts = load_data(dirname)
    sequences = tokenizer.texts_to_sequences(texts)
    
    while True:
        # Shuffle the data
        indices = np.arange(len(images))
        np.random.shuffle(indices)
        images = images[indices]
        sequences = [sequences[i] for i in indices]
        
        # Calculate the split point for validation data (20%)
        split_point = int(0.2 * len(images))
        val_images, val_sequences = images[:split_point], sequences[:split_point]
        
        for start in range(split_point, len(images), batch_size):
            end = min(start + batch_size, len(images))
            
            X_batch, y_batch, image_data_batch = preprocess_data(sequences[start:end], images[start:end])
            
            yield [image_data_batch, X_batch], y_batch


In [10]:
# Assuming you have already defined your val_generator
# with the appropriate parameters (dirname and batch_size)

# Get the validation data generator
val_gen = val_generator(dirname='1190Data/', batch_size=32)

# Calculate the total number of validation samples
total_val_samples = len([3])  # Assuming val_sequences contains your validation data

# Calculate the number of batches
num_batches = total_val_samples // 32

# Print the results
print(f"Total validation samples: {total_val_samples}")
print(f"Number of batches: {num_batches}")
print(f"Batch size: {32}")


Total validation samples: 1
Number of batches: 0
Batch size: 32


In [17]:
# len(next(val_generator(dirname='1190Data/', batch_size=32))[0][1])

2288

In [5]:

# Create the encoder
image_model = Sequential()
image_model.add(Conv2D(16, (3, 3), padding='valid', activation='relu', input_shape=(128, 128, 3,)))
image_model.add(Conv2D(16, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

image_model.add(Flatten())
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))

image_model.add(RepeatVector(max_length))

visual_input = Input(shape=(128, 128, 3,))
encoded_image = image_model(visual_input)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(128, return_sequences=True)(language_model)
language_model = LSTM(128, return_sequences=True)(language_model)

# Create the decoder
decoder = concatenate([encoded_image, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[visual_input, language_input], outputs=decoder)
optimizer = RMSprop(learning_rate=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [ ]:

# batch_size = 32
# train_generator = data_generator(dirname=dirnamee, batch_size=batch_size)
# steps_per_epoch = len(train_sequences) // batch_size

# # Define the checkpoint
# from keras.callbacks import ModelCheckpoint
# checkpoint_path = "model_weights_epoch_{epoch:02d}.h5"
# checkpoint = ModelCheckpoint(checkpoint_path, 
#                              save_weights_only=True, 
#                              save_freq='epoch', 
#                              period=2)

# # Fit the model with validation data
# model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, validation_data=val_generator)


In [18]:

# Train the model with batches
batch_size = 32
train_generator = data_generator(dirname=dirnamee, batch_size=batch_size)
val_generator = val_generator(dirname='1190Data/', batch_size=batch_size)
steps_per_epoch = len(train_sequences) // batch_size


In [11]:
# Define the checkpoint
from keras.callbacks import ModelCheckpoint
checkpoint_path = "model_weights_epoch_{epoch:02d}.h5"
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_freq='epoch', 
                             period=2)

In [19]:

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, validation_data=val_generator)


C:\Users\nitish.raghav\AppData\Local\Temp\ipykernel_15196\2209798556.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, validation_data=val_generator)


Epoch 1/10

40/54 [=====================>........] - ETA: 25:56 - loss: 2.3863

: 